In [2]:
import torch as t
from torch.autograd import Variable

a = Variable(t.randn(3,4))
b = Variable(t.randn(3,4))

In [3]:
print(a,b,sep='\n')
a.add(b)
print(a,b,sep='\r')

tensor([[-0.3532, -0.0329,  0.4700,  2.5445],
        [-0.9652, -1.2532,  0.1932, -0.3219],
        [ 1.3640, -0.9994,  0.6431, -1.0805]])
tensor([[-0.5821, -0.7738, -1.3959, -1.6169],
        [-0.6826,  0.6801, -1.5474, -2.3205],
        [-0.0719,  0.9943,  0.6800, -1.2078]])
tensor([[-0.3532, -0.0329,  0.4700,  2.5445],
        [-0.9652, -1.2532,  0.1932, -0.3219],
tensor([[-0.5821, -0.7738, -1.3959, -1.6169],
        [-0.6826,  0.6801, -1.5474, -2.3205],
        [-0.0719,  0.9943,  0.6800, -1.2078]])


## 1.张量的基本运算
pytorch中使用三种方式进行基本运算：
1. 直接使用符号
```python
c = a + b
```
2. 使用torch.add
```python
   c = torch.add(a,b)
```
3. 使用a.add
```python
   c = a.add(b)
```
注意，这种方式相加的时候，a不会随之改变，如果需要随之改变，使用
```python
   c = a.add_(b)
   torch.add(a, b, result=c)
```

torch支持许多类似的操作，如果不支持的话，可以先转化为numpy进行操作，再转化
回tensor

方法是：
使用tensor.numpy()

从numpy转为tensor时使用torch.from_numpy(a)返回一个tensor。这两者是共用内存的

In [4]:
grad_test = Variable(t.ones(2,2), requires_grad=True)
total = 2*grad_test.sum()
total.backward(retain_graph=True)
print(grad_test.grad)
print(grad_test.data)

tensor([[2., 2.],
        [2., 2.]])
tensor([[1., 1.],
        [1., 1.]])


## 2.自动求导
每一个Variable类内部有一个包含数据的tensor:data，还有一个用于保存梯度的tensor：grad。

在定义了张量之间的运算后，可以通过调用**结果的backward()方法**来使与其有关的张量上的grad

梯度被计算出来


In [5]:
unsqueeze,unsqueeze_ = grad_test.unsqueeze(2),grad_test.unsqueeze(1)
print(unsqueeze)
print(unsqueeze_)

tensor([[[1.],
         [1.]],

        [[1.],
         [1.]]], grad_fn=<UnsqueezeBackward0>)
tensor([[[1., 1.]],

        [[1., 1.]]], grad_fn=<UnsqueezeBackward0>)


## 3.使用unsqueeze来增加一个维度
unsqueeze将会在指定张量的指定的轴上增加一个维度，例如：
```python
    a = a.unsqueeze(1)
```
将会在a向量的第二维（序号为一）的维度上增加一个维度，即用一个维度包裹原来的该维度

同理，可以使用squeeze来压缩一个维度

如果不在squeeze中给定参数，将会把所有为1维的维度压缩掉

In [6]:
#源数据
inputs = t.Tensor([[1,2,3],[4,5,6],[7,8,9]])
#指定选择的下标
indexes = t.LongTensor([0,2])
#指定不同的维度上进行选择
print(inputs.index_select(0, indexes))
print(inputs.index_select(1, indexes))

tensor([[1., 2., 3.],
        [7., 8., 9.]])
tensor([[1., 3.],
        [4., 6.],
        [7., 9.]])


In [7]:
print(inputs.masked_select(inputs>5))

tensor([6., 7., 8., 9.])


## 4.masked_select
利用二进制掩码序列，找到1对应的位置的值作为一维张量返回

上例中，inputs>5 产生了一个与原张量形状相同，但是只有对应位置上的值大于5才会为1否则为0的张量

利用这个掩码张量，返回的所有位置为1的值便是在返回所有大于5的值，以一维张量的形式

In [8]:
gather_diag = t.LongTensor([[0,1,2]])
print(inputs.gather(0, gather_diag))
print(inputs.scatter(0, gather_diag, t.Tensor([[-1,-1,-1]])))

tensor([[1., 5., 9.]])
tensor([[-1.,  2.,  3.],
        [ 4., -1.,  6.],
        [ 7.,  8., -1.]])


## 5.gather与scatter
gather聚集函数利用一个index向量在指定的轴上，来获取源输入中的部分向量组成新张量，输入张量的大小与index一致

获取的方式是：
```python
    out[i][j] = input[index[i][j]][j] #dim=0
    out[i][j] = input[i][index[i][j]] #dim=1
```
即：指定的dim是index作用的域，其他轴都遍历。利用的坐标与输出的坐标位置相同

scatter作用与gather刚好相反，将指定的值按轴以index指定的形式放回指定张量中

In [9]:
#高级索引
#高级索引不会共享内存
print(inputs)
print(inputs[[0,2],[0,2]])
print(inputs[[0,2],[1]])
print(inputs[...,[0,1]])

tensor([[1., 2., 3.],
        [4., 5., 6.],
        [7., 8., 9.]])
tensor([1., 9.])
tensor([2., 8.])
tensor([[1., 2.],
        [4., 5.],
        [7., 8.]])


In [10]:
print(inputs.sum(dim=1, keepdim=True))
print(inputs)

tensor([[ 6.],
        [15.],
        [24.]])
tensor([[1., 2., 3.],
        [4., 5., 6.],
        [7., 8., 9.]])


## 6.使用归并操作
sum，mean，norm，std等操作能够将输入的一些值进行约减，关键在于要指定轴

如果一个i,j,k的矩阵指定dim=0，则输出为1,j,k。即指定的dim就是最终为1的轴，其对应的另外一些轴上的值将被约减。

例如指定dim=1，则相当于dim=0上的值被约减

In [11]:
compare = t.ones((3,3)) + 4
#直接对同尺寸大小的张量进行比较操作，将会得到一个逐元素比较得到的byte矩阵
print(inputs>compare)
#使用这个byte矩阵可以从原矩阵中获得对应位置的数据
print(inputs[inputs>compare])
print(inputs.max())
#max的dim指定的是比较的维度，这个维度就是显式的维度
#返回值是一个元组，第一个元素是最大值，第二个是下标
print(inputs.max(dim=0))
#获得两个张量中，每个位置最大元素组成的新张量
print(inputs.max(compare))
#还可以使用clamp来指定一个固定的最大/最小值
print(inputs.clamp(min=5))

tensor([[0, 0, 0],
        [0, 0, 1],
        [1, 1, 1]], dtype=torch.uint8)
tensor([6., 7., 8., 9.])
tensor(9.)
(tensor([7., 8., 9.]), tensor([2, 2, 2]))
tensor([[5., 5., 5.],
        [5., 5., 6.],
        [7., 8., 9.]])
tensor([[5., 5., 5.],
        [5., 5., 6.],
        [7., 8., 9.]])


In [12]:
print(inputs.t())
print(inputs.t().is_contiguous())
print(inputs.t().contiguous())
print(inputs.transpose(0,1))

tensor([[1., 4., 7.],
        [2., 5., 8.],
        [3., 6., 9.]])
False
tensor([[1., 4., 7.],
        [2., 5., 8.],
        [3., 6., 9.]])
tensor([[1., 4., 7.],
        [2., 5., 8.],
        [3., 6., 9.]])


## 7.矩阵转置
torch.t()和torch.transpose()都可以实现矩阵的转置，不同的是，前者只能转置2D矩阵，而后者可以转置任意维度

的矩阵

transpose需要两个参数，即两个需要交换的维度的下标

t方法制造的矩阵不是连续的，需要使用contiguous方法变为连续的

In [13]:
test = t.Tensor([[1],[2],[3]])
print(inputs.repeat(2,2))   #将各个维度上扩大多少倍
print(test.expand(3,3))     #将张量扩张为何种大小的张量

tensor([[1., 2., 3., 1., 2., 3.],
        [4., 5., 6., 4., 5., 6.],
        [7., 8., 9., 7., 8., 9.],
        [1., 2., 3., 1., 2., 3.],
        [4., 5., 6., 4., 5., 6.],
        [7., 8., 9., 7., 8., 9.]])
tensor([[1., 1., 1.],
        [2., 2., 2.],
        [3., 3., 3.]])


## 8.repeat和expand
两者都是可以将张量内部的值进行重复化，不同的是，expand只能对一个含有单一值的维度进行扩张

但是repeat没有这个限制。前者更加节省内存而已。

参数：
repeat：各个维度上重复多少次

expand：指定各个维度的大小，可以使用-1来自适应

In [14]:
grad_a = t.ones((2,2), requires_grad=True)
grad_b = t.randn((2,2))
grad_c = 2*grad_a**2 + grad_b
grad_d = (grad_c**2).sum()
grad_d.backward()
print(grad_a.grad)
#手动求导的结果
print(2*grad_c*4*grad_a)

tensor([[16.3351, 18.6820],
        [28.8146, 22.3942]])
tensor([[16.3351, 18.6820],
        [28.8146, 22.3942]], grad_fn=<MulBackward0>)


## 9.使用自动求导机制
1. 直接创建tensor而不是Variable，因为后者已经被废除。在需要计算向量的tensor的定义处加上requires_grad=True
2. 尽管有些变量没有声音需要梯度，但是由于有些需要梯度的张量的计算依赖于它，所以还是会计算张量，但是之后就被丢弃
3. 需要从计算末尾进行链式法则的反向求导。注意：结果必须是一个标量scalar。其实也可以从中间的向量开始反向传播，但是需要指定目标函数对其的梯度向量，可以理解为梯度反向传播时，中间节点在已知本节点的梯度情况下向后传播梯度。方法是：指定grad_variables。由于目标函数一般是一个标量，因此grad_variables的形状与中间节点的形状一致
4. 如果一个变量不需要求导，且路径上依赖这个变量的变量（子变量）都不需要求导，则可以设置volatile=True。而且volatile的优先级高于requires_grad。


示例中，手动求导结果和backward()结果相同


In [15]:
#a为叶节点，没有操作函数
print(grad_a.grad_fn)
#c是一个中间节点，包含多个操作函数，因为是多项式
print(grad_c.grad_fn)
#展示c中的所有操作函数
print(grad_c.grad_fn.next_functions)

None
((<MulBackward0 object at 0x000002D110ACC550>, 0), (None, 0))



## 10.自动求导细则
在底层的实现中，每一个tensor都包含一个实例grad和一个操作实例grad_fn，前者用于保存某一次backward()时计算得到的梯度，后者用于记录对应的操作来计算特定的梯度。在每一次backward()进行之后，需要梯度的节点会保留梯度在grad中，但是不需要梯度的中间变量即使会在backwar()过程中计算梯度，但是会在计算结束后删除掉。如果要获取中间变量的梯度，有两种方式：
1. 使用hook:自定义一个hook的函数，输入是梯度，没有输出值。将这个hook利用register_hook()注册到对应的变量上，使用完以后移除
```python
    def hook(grad):
        print(grad)
    handle_hook = y.register_hook(hook)
    #...
    handle_hook.remove()
```
2. 使用autograd.grad方法，重新计算grad:
```python
    grad = t.autograd.grad(z, y) #z对y求偏导
```

如果遭遇不能自动求导的操作，则需要自己写出自定义的autograd.Function来对应。使用的时候，使用类的apply方法
    
        

In [16]:
class MulSqureAdd(t.autograd.Function):
    #两个方法：forward和backward的输入互相是对方的输出，输出是互相的输入
    
    @staticmethod
    def forward(ctx, w, x, b, x_grad=True):
        #参数:ctx是一个固定参数，w是系数，b是偏置
        #保存是否需要对x求导的指示变量
        ctx.x_grad = x_grad
        #将到时候backward()时需要的变量w和x储存起来
        ctx.save_for_backward(w,x)
        return w*w*x+b
    
    @staticmethod
    #由于forward一个输出，因此一个输入
    #这个grad_output是目标函数值传递到下一层时的梯度，本节点的梯度需要乘该值
    def backward(ctx, grad_output):
        #读取保存的变量
        x_grad = ctx.x_grad
        #从saved_tensors中读取利用save_for_backward()储存的变量
        w,x = ctx.saved_tensors
        grad_w = grad_output*x*w*2
        if x_grad:
            grad_x = grad_output*w*w
        #对于不需要求导的变量返回None1
        else:
            grad_x = None
        grad_b = grad_output * 1 #因为b的系数为1
        return grad_w,grad_x,grad_b,None   #最后一个None是因为还有一个指示是否对x求导的变量，无法求导而为None
    
grad_1_a = t.ones((2,2), requires_grad=True)
grad_1_b = t.randn((2,2))
grad_1_c = t.rand((2,2))
grad_1_d = MulSqureAdd.apply(grad_1_b, grad_1_a, grad_1_c).sum()    #用apply来使用自定义的操作函数
grad_1_d.backward()
#自动求导和手动求导结果相同
print(grad_1_a.grad)
print(grad_1_b*grad_1_b)

tensor([[2.3515, 0.1299],
        [0.8858, 5.1850]])
tensor([[2.3515, 0.1299],
        [0.8858, 5.1850]])


## 11.编写网络层
所有模型都继承自module，而且内部又包含了一些子module，这些module就是层。这些module的编写需要注意一下几个方面：
1. 这些作为网络层的子模块也需要继承自nn.Module，而且需要覆盖父类的init方法
2. 在init中需要定义可以学习的参数，并且封装为Parameter。这些Parameter是默认需要进行梯度的
3. 需要实现forward()方法，不需要实现backward()方法
4. 使用层时，直接使用layer(input)而不是layer.forward(input)，因为可能会有hook作用

在实现了这些网络层以后，就可以利用网络层来搭建模型。这些网络层中被封装为Parameter的参数也会作为模型整体的Parameter

**注意：在调用自带的layer时，由于这些layer已经默认需要接受的是一个batch的数据，因此在输入数据只有一个的时候需要使用unsqueeze来将输入伪装为batch_size为1的数据**

In [28]:
#总结：序列长度为2，输入维度大小为3，批大小为3，隐藏层维度大小为3，方向为1，层数为1

#参数:输入的特征维度为3,隐藏层的宽度为3，层数为1
lstm = t.nn.LSTM(3, 3, 1)

#隐藏层的初始状态定义
#参数:层数x方向数量=1，批大小为3，隐藏层的宽度为3
h_0 = t.randn((1,3,3))

#状态层cell的初始化定义
#参数同上
c_0 = t.randn((1,3,3))

#输入的大小定义
#参数:序列长度为2，批大小为3，输入的特征维度为3
Input = t.randn((2,3,3))

#将序列输入到LSTM中
#先写输入，再将初始的隐藏层和状态层以元组方式打包送入LSTM
#返回值：
#1.一个序列长度的序列输出，形状为：序列大小，批大小，方向数*隐藏层维度大小
#2.一个元组，第一个元素是序列长度n以后的隐藏层状态h_n,第二个元素是序列长度n以后的状态值c_n
#  两者的形状都是：层数，批大小，隐藏层维度大小
output,state = lstm(Input, (h_0, c_0))
print('输出: \n',output)
print('序列结束后的h_x:\n ', state[0])
print('序列结束后的c_x: \n', state[1])

输出: 
 tensor([[[ 0.0247,  0.3816,  0.2296],
         [ 0.2003, -0.3982, -0.1844],
         [ 0.1391,  0.0180,  0.2558]],

        [[ 0.1839,  0.4237, -0.0451],
         [ 0.3742, -0.7128, -0.1011],
         [ 0.2010,  0.0451, -0.0277]]], grad_fn=<StackBackward>)
序列结束后的h_x:
  tensor([[[ 0.1839,  0.4237, -0.0451],
         [ 0.3742, -0.7128, -0.1011],
         [ 0.2010,  0.0451, -0.0277]]], grad_fn=<StackBackward>)
序列结束后的c_x: 
 tensor([[[ 0.3008,  0.6199, -0.0909],
         [ 0.4638, -1.0984, -0.2470],
         [ 0.3390,  0.0616, -0.0506]]], grad_fn=<StackBackward>)


## 12.如果使用的是 LSTM Cell
可以使用Cell来代替循环的LSTM，相当于每一个Cell就是一个时间序列处的模型，则：
```python
#只需要写输入维度与隐藏层维度，不需要写层数
lstm = t.nn.LSTMCell(3,3)
#只写批大小和隐藏层宽度
h_x = t.randn((3,3))
c_x = t.randn((3,3))
#输入依旧是：序列长度，批大小，输入维度
iinput = t.randn((2,3,3))
#遍历输入序列
for each_input in iinput:
    #将上一轮的隐藏层状态和门控状态作为本轮的输入，同时更新这两个状态
    #h_x隐藏层状态其实就是模型输出
    h_x,c_x = lstm(each_input, (h_x,c_x))
    print(h_x)
```

## 13.nn.Functional和nn.Module
- 每一个Module对应的功能都有一个Functional函数对应之，不同的是，前者可以直接提取可学习的参数，后者不行。因此有需要学习的参数的时候（例如在init中定义层数时），使用nn.Module，在forward中使用激活函数，池化层和dropout等没有参数的层时，使用nn.Functional。

- 也可以使用Functional来构建需要学习参数的层，但是需要将其使用Parameter参数化，因此需要重新定义一个继承Module的类，在init方法中使用Parameter参数化，在forward方法中使用nn.Functional。

## 14.初始化策略
- 初始化的策略包含在torch.nn.init中，使用时，直接调用init.(name)(input, parameters)，则初始化器将会对input，利用参数来初始化
- 在初始化的时候，有两者方式可以进行统一的初始化：
    1. 对Module中的named_parameters进行遍历，使用if语句检查name来对每个不同的参数俩优化
    2. 对模型Module使用apply传递进一个初始化函数，这个初始化函数将会作用于Module中的每一个子Module。初始化函数中可以对子Module的name进行检查来定制初始化策略

## 15.nn.Module分析
   - Module中包含_parameters，_modules，train等属性，用于保存本模型的参数，子模型等。需要注意的是，只有在本Module内使用nn.Parameter显式包装了的参数才会在_parameters中，在本模型中定义的子模型的会进入_modules，但是其参数不会进入到_parameters中
   - 如果要获取本模型的中包含子模型的所有参数，应该使用parameters()方法或者named_parameters()方法
   - train属性是用于对一些在训练阶段和测试阶段行为不一致的层进行协调用的，例如dropout，可以对模型直接调用train()方法或者eval()方法来进入训练或者测试模型来一次性设定该参数
   - 在外部，给模型新增实例域 有两种情况，第一种是新添加的是nn.Modules或者nn.parameter，则会将其添加到对应的字典中，其他的对象则直接新增加一个实例域
   - 保存模型的参数:使用torch.save(model.state_dict(), path)的方式，将model的参数进行保存。加载时，使用torch.load()将参数读入一个已经定义好的相同的模型中即可